This example is an experiment to validate that indeed we get better MSE using the real noise rather then the estimated one.
It's a sort of a sanity check for the algorithm.

In [1]:
from src.motion_models.trajectory_generation.route_generation import *
from src.visual.visual_tools import *

import numpy as np

%load_ext autoreload
%autoreload 2

## CV Model ##

In [2]:
from src.filters import InitializationData, CVKalmanFilter

REAL_NOISE = np.array([0.2, 0.2])
REAL_PROCESS_NOISE_STD = np.array([0.5,0.5])
dt=0.04
white_accel_density = 1e1
T = 200
train_seed = 44
test_seed = 45

initial_velocity = np.array([14,5])
initial_position = np.array([1,5])

# Generate trajectories with the real noise
initial_trajectory_state = TrajectoryState(position=initial_position, velocity=initial_velocity)
trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=initial_trajectory_state, number_of_trajectories=40, measurement_noise_std=REAL_NOISE, vel_change_std=REAL_PROCESS_NOISE_STD, seed=train_seed)

# Initialize the noise std with large unkonwn value
INITIAL_NOISE = np.array([10, 10])
INITIAL_PROCESS_NOISE_STD = np.array([10,10])

# Initialzing the KF
initial_matrices_data = InitializationData(observation_noise_std=INITIAL_NOISE, process_noise_std=INITIAL_PROCESS_NOISE_STD)
estimated_kf = CVKalmanFilter(2, 4, 2, dt, initial_matrices_data)
estimated_kf.initialize()

# R fitting with fixed old_em method
print("Estimating R")
print(f"True R should be: {np.diag(REAL_NOISE**2)}")
print(f"Initial R: {estimated_kf.kf.R}")

for i in range(40):
    traj = trajectories[i][0]
    for j in range(3):
        # Estimate R
        res = estimated_kf.em(measurements=traj, n_iter=1, estimate_Q=False, estimate_R=True)
    print(f"R : {estimated_kf.kf.R}")

print("Estimating Q")
for i in range(40):
    traj = trajectories[i][0]
    
    for j in range(20):
        # Estimate Q
        res = estimated_kf.em(measurements=traj, n_iter=1, estimate_Q=True, estimate_R=False)
    print(f"Q : {estimated_kf.kf.Q}")


estimated_Q = estimated_kf.kf.Q
estimated_R = estimated_kf.kf.R
#plot_trajectories_with_predictions_shifted(noisy=test_traj, clean=clean_test_traj, predictions=[(preds, 1,)])


Estimating R
True R should be: [[0.04 0.  ]
 [0.   0.04]]
Initial R: [[100   0]
 [  0 100]]
R : [[ 7.47357474e+01 -9.40217678e-04]
 [-9.40217678e-04  7.47356529e+01]]
R : [[ 5.59628772e+01 -3.69101153e-05]
 [-3.69101153e-05  5.59619630e+01]]
R : [[4.19968768e+01 4.09113382e-03]
 [4.09113382e-03 4.19953562e+01]]
R : [[3.15845233e+01 3.21096277e-03]
 [3.21096277e-03 3.15830527e+01]]
R : [[2.38102774e+01 3.54518610e-03]
 [3.54518610e-03 2.38090781e+01]]
R : [[1.79955174e+01 3.51608660e-03]
 [3.51608660e-03 1.79955055e+01]]
R : [[1.36382898e+01 2.31129565e-03]
 [2.31129565e-03 1.36381725e+01]]
R : [[1.03701595e+01 1.26384001e-03]
 [1.26384001e-03 1.03682238e+01]]
R : [[7.90789926e+00 1.38795738e-03]
 [1.38795738e-03 7.90628665e+00]]
R : [[6.05084390e+00 7.52420689e-04]
 [7.52420689e-04 6.05124561e+00]]
R : [[ 4.64690764e+00 -2.73825254e-04]
 [-2.73825254e-04  4.64758504e+00]]
R : [[3.58305528e+00 2.78646303e-04]
 [2.78646303e-04 3.58277016e+00]]
R : [[ 2.77543391e+00 -1.60085014e-04]
 [-1.

In [3]:
initial_trajectory_state = TrajectoryState(position=initial_position, velocity=initial_velocity)
test_trajectories = generate_cv_trajectory(T=T, dt=dt, initial_state=initial_trajectory_state, number_of_trajectories=40, measurement_noise_std=REAL_NOISE, vel_change_std=REAL_PROCESS_NOISE_STD, seed=test_seed)

real_kf_mses = []
estimated_kf_mses = []

real_initial_matrices_data = InitializationData(observation_noise_std=REAL_NOISE, process_noise_std=REAL_PROCESS_NOISE_STD)
real_kf = CVKalmanFilter(2, 4, 2, dt, real_initial_matrices_data)
real_kf.initialize()

for noisy, clean, _ in test_trajectories:
    print("========")
    est_mse = estimated_kf.evaluate_on_trajectory(noisy, clean, normalize_mse=False)
    real_mse = real_kf.evaluate_on_trajectory(noisy, clean, normalize_mse=False)
    print(f"Estimated MSE: {est_mse}")
    print(f"Real MSE:      {real_mse}")
    real_kf_mses.append(real_mse)
    estimated_kf_mses.append(est_mse)
    
    real_kf.initialize()
    estimated_kf.initialize()
    estimated_kf.kf.R = estimated_R
    estimated_kf.kf.Q = estimated_Q

print("Real MSE mean:")
print(np.mean(np.array(real_kf_mses)))

print("Estimated MSE mean:")
print(np.mean(np.array(estimated_kf_mses)))


Estimated MSE: 0.031091373428677366
Real MSE:      0.0309184909242686
Estimated MSE: 0.024258176424122757
Real MSE:      0.024213248336099644
Estimated MSE: 0.026154420642595617
Real MSE:      0.025872559827778772
Estimated MSE: 0.0196951937069345
Real MSE:      0.019666293494048935
Estimated MSE: 0.02204133946591343
Real MSE:      0.021999231901290987
Estimated MSE: 0.021324605317201616
Real MSE:      0.021297640771851957
Estimated MSE: 0.024798038917533582
Real MSE:      0.024594247678427508
Estimated MSE: 0.023147382776417084
Real MSE:      0.023316519435213682
Estimated MSE: 0.02559905260396336
Real MSE:      0.025345959022507938
Estimated MSE: 0.02097568566954922
Real MSE:      0.020933819951199768
Estimated MSE: 0.021396566277789332
Real MSE:      0.021286240279429738
Estimated MSE: 0.037768286902741935
Real MSE:      0.03705715596062838
Estimated MSE: 0.027647842480406137
Real MSE:      0.027375468755351632
Estimated MSE: 0.026900465754688754
Real MSE:      0.026537413489118835


## CA Model ##

In [4]:
from src.filters import InitializationData, CAKalmanFilter

dt = 0.04
REAL_NOISE = np.array([0.4,0.4])
white_accel_density = 5
real_accel_noise_std = np.sqrt(dt * white_accel_density)
initial_acceleration = np.array([1, 1])
initial_velocity = np.array([3,5])
initial_position = np.array([0,0])
T = 200

initial_trajectory_state = TrajectoryState(position=initial_position, velocity=initial_velocity, acceleration=initial_acceleration)
trajectories = generate_ca_trajectory(T=T, dt=dt,initial_state=initial_trajectory_state, measurement_noise_std=REAL_NOISE, number_of_trajectories=40, seed=train_seed, accel_noise_std=real_accel_noise_std)

# Initialize the noise std with large unkonwn value
INITIAL_NOISE = np.array([10, 10])
INITIAL_WHITE_ACCEL_DENSITY = 10

# Initialzing the KF
initial_matrices_data = InitializationData(observation_noise_std=INITIAL_NOISE, white_accel_density=INITIAL_WHITE_ACCEL_DENSITY)
estimated_kf = CAKalmanFilter(2, 6, 2, dt, initial_matrices_data)
estimated_kf.initialize()

# R fitting with fixed old_em method
print("Estimating R")
print(f"True R should be: {np.diag(REAL_NOISE**2)}")
print(f"Initial R: {estimated_kf.kf.R}")

for i in range(40):
    traj = trajectories[i][0]
    for j in range(3):
        # Estimate R
        res = estimated_kf.em(measurements=traj, n_iter=1, estimate_Q=False, estimate_R=True)
    print(f"R : {estimated_kf.kf.R}")

print("Estimating Q")
for i in range(40):
    traj = trajectories[i][0]
    
    for j in range(20):
        # Estimate Q
        res = estimated_kf.em(measurements=traj, n_iter=1, estimate_Q=True, estimate_R=False)
    print(f"Q : {estimated_kf.kf.Q}")


estimated_Q = estimated_kf.kf.Q
estimated_R = estimated_kf.kf.R
#plot_trajectories_with_predictions_shifted(noisy=test_traj, clean=clean_test_traj, predictions=[(preds, 1,)])

print(estimated_R)
print(estimated_Q)

Estimating R
True R should be: [[0.16 0.  ]
 [0.   0.16]]
Initial R: [[100   0]
 [  0 100]]
R : [[7.35939121e+01 4.20830795e-02]
 [4.20830795e-02 7.34667591e+01]]
R : [[54.30070905  0.13505814]
 [ 0.13505814 54.05512535]]
R : [[40.07663064  0.21608769]
 [ 0.21608769 39.84281869]]
R : [[29.51528928  0.13463471]
 [ 0.13463471 29.36948059]]
R : [[21.74849215  0.10942063]
 [ 0.10942063 21.65172897]]
R : [[16.03963633  0.07751382]
 [ 0.07751382 16.02537146]]
R : [[11.85540577  0.08329835]
 [ 0.08329835 11.88608908]]
R : [[8.79685519 0.06633386]
 [0.06633386 8.80053993]]
R : [[6.52472292 0.04243411]
 [0.04243411 6.54914937]]
R : [[4.8711919  0.03278063]
 [0.03278063 4.87420019]]
R : [[3.64817258 0.02732298]
 [0.02732298 3.64025405]]
R : [[2.73267298 0.02244304]
 [0.02244304 2.72807453]]
R : [[2.06644433 0.01540325]
 [0.01540325 2.04684076]]
R : [[1.57003099 0.01177625]
 [0.01177625 1.54935427]]
R : [[1.23645607 0.02319267]
 [0.02319267 1.19160344]]
R : [[0.95363961 0.01861856]
 [0.01861856 0

In [5]:
initial_trajectory_state = TrajectoryState(position=initial_position, velocity=initial_velocity)
test_trajectories = generate_ca_trajectory(T=T, dt=dt,initial_state=initial_trajectory_state, measurement_noise_std=REAL_NOISE, number_of_trajectories=40, seed=test_seed, accel_noise_std=real_accel_noise_std)


real_kf_mses = []
estimated_kf_mses = []

real_initial_matrices_data = InitializationData(observation_noise_std=REAL_NOISE, white_accel_density=white_accel_density)
real_kf = CAKalmanFilter(2, 6, 2, dt, real_initial_matrices_data)
real_kf.initialize()

for noisy, clean, _ in test_trajectories:
    print("========")
    est_mse = estimated_kf.evaluate_on_trajectory(noisy, clean, normalize_mse=False)
    real_mse = real_kf.evaluate_on_trajectory(noisy, clean, normalize_mse=False)
    print(f"Estimated MSE: {est_mse}")
    print(f"Real MSE:      {real_mse}")
    real_kf_mses.append(real_mse)
    estimated_kf_mses.append(est_mse)
    
    real_kf.initialize()
    estimated_kf.initialize()
    estimated_kf.kf.R = estimated_R
    estimated_kf.kf.Q = estimated_Q

print("Real MSE mean:")
print(np.mean(np.array(real_kf_mses)))

print("Estimated MSE mean:")
print(np.mean(np.array(estimated_kf_mses)))


Estimated MSE: 0.06612369497221286
Real MSE:      0.06663755490341941
Estimated MSE: 0.060307920174900595
Real MSE:      0.057128191912846175
Estimated MSE: 0.054172257331348866
Real MSE:      0.05234179583384505
Estimated MSE: 0.06177619057847002
Real MSE:      0.06065967820933881
Estimated MSE: 0.04635273085300966
Real MSE:      0.041453307049513936
Estimated MSE: 0.09547753441650336
Real MSE:      0.09428188058123656
Estimated MSE: 0.05344827079077471
Real MSE:      0.0550199263196121
Estimated MSE: 0.08307415920395916
Real MSE:      0.08192269437960543
Estimated MSE: 0.13158097022188825
Real MSE:      0.13078588067002525
Estimated MSE: 0.06248842158548315
Real MSE:      0.06252965831401391
Estimated MSE: 0.06729302031891994
Real MSE:      0.06416281262501526
Estimated MSE: 0.06465793589336741
Real MSE:      0.06716355807747634
Estimated MSE: 0.05801222332243963
Real MSE:      0.05468993358771295
Estimated MSE: 0.06852905815590225
Real MSE:      0.06923936202974527
Estimated MSE: 0.